In [114]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [40]:
df = pd.read_excel("05.2022_Выручка.xlsx")
df.head()

,TimeThirty,DAT,Торговая точка,Касса,orgtype,terminal,tzone,TotalSumm
0,2022-05-01 00:00:00.000,2022-05-01,Торговая точка 0601,7683,Общепит,B,Чистая зона ВВЛ-вылет,755 448
1,2022-05-04 13:30:00.000,2022-05-04,Торговая точка 2373,3114,Общепит,B,Общедоступная зона-вылет,46 800
2,2022-05-05 18:00:00.000,2022-05-05,Торговая точка 8998,9887,Торговля,B,Чистая зона ВВЛ-вылет,160 160
3,2022-05-25 22:00:00.000,2022-05-25,Торговая точка 8313,2251,Общепит,B,Чистая зона ВВЛ-вылет,248 960
4,2022-05-10 00:30:00.000,2022-05-10,Торговая точка 8998,9887,Торговля,B,Чистая зона ВВЛ-вылет,117 600


In [41]:
for col in df.columns:
    print(col, df[col].unique(), "\n")

TimeThirty ['2022-05-01 00:00:00.000' '2022-05-04 13:30:00.000'
 '2022-05-05 18:00:00.000' ... '2022-05-24 03:00:00.000'
 '2022-05-11 02:00:00.000' '2022-05-14 02:00:00.000'] 

DAT ['2022-05-01' '2022-05-04' '2022-05-05' '2022-05-25' '2022-05-10'
 '2022-05-17' '2022-05-15' '2022-05-14' '2022-05-29' '2022-05-03'
 '2022-05-09' '2022-05-23' '2022-05-30' '2022-05-22' '2022-05-31'
 '2022-05-02' '2022-05-24' '2022-05-08' '2022-05-16' '2022-05-21'
 '2022-05-26' '2022-05-27' '2022-05-20' '2022-05-07' '2022-05-28'
 '2022-05-19' '2022-05-12' '2022-05-06' '2022-05-13' '2022-05-18'
 '2022-05-11'] 

Торговая точка ['Торговая точка 0601' 'Торговая точка 2373' 'Торговая точка 8998'
 'Торговая точка 8313' 'Торговая точка 2968' 'Торговая точка 5648'
 'Торговая точка 3726' 'Торговая точка 8219'] 

Касса [7683 3114 9887 2251 9033 6213 3601  538 7998  279 5944 1290] 

orgtype ['Общепит' 'Торговля'] 

terminal ['B'] 

tzone ['Чистая зона ВВЛ-вылет' 'Общедоступная зона-вылет'
 'Чистая зона ВВЛ-прилет'] 

To

In [64]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    data = df.copy()

    data["TotalSumm"] = data["TotalSumm"].apply(lambda x : x.replace("\xa0", ""))
    data["TotalSumm"] = data["TotalSumm"].apply(lambda x : x.replace(",", "."))
    data["TotalSumm"] = data["TotalSumm"].astype(float)

    data["Торговая точка"] = data["Торговая точка"].apply(lambda x: x.replace("Торговая точка ", ""))

    data["TimeThirty"] = pd.to_datetime(data["TimeThirty"])
    data["Касса"] = data["Касса"].astype(object)

    data.drop(columns=["terminal", "DAT"], inplace=True)

    data.rename(columns={
        "TimeThirty": "time", "Торговая точка": "sell_point", 
        "Касса": "cashier", "tzone": "zone", "TotalSumm": "revenue"
    }, inplace=True)

    data = data.set_index(["sell_point", "time"]).reset_index()
    data = data.sort_values(by=["sell_point", "time"])

    return data

In [118]:
def draw_ts(data: pd.DataFrame, target: str) -> None:
    sellpoints = data["sell_point"].unique()
    for i in range(len(sellpoints)):
        sp_data = data[data["sell_point"] == sellpoints[i]]
        cashiers = sp_data["cashier"].unique()
        for j in range(len(cashiers)):
            spc_data = sp_data[sp_data["cashier"] == cashiers[j]].copy()
            x = spc_data["time"]
            y = spc_data[target]
            plt.figure(figsize=(20, 5))
            plt.plot(x, y)
            plt.title(f"Sellpoint {sellpoints[i]}, cashier {cashiers[j]}")

In [65]:
data = preprocess(df)
data

,sell_point,time,cashier,orgtype,zone,revenue
0,0601,2022-05-01 00:00:00,7683,Общепит,Чистая зона ВВЛ-вылет,755448.0
2696,0601,2022-05-01 00:30:00,7683,Общепит,Чистая зона ВВЛ-вылет,1298616.0
2573,0601,2022-05-01 01:00:00,7683,Общепит,Чистая зона ВВЛ-вылет,451200.0
11515,0601,2022-05-01 01:30:00,7683,Общепит,Чистая зона ВВЛ-вылет,276000.0
3370,0601,2022-05-01 02:30:00,7683,Общепит,Чистая зона ВВЛ-вылет,68000.0
...,...,...,...,...,...,...
8419,8998,2022-05-31 22:30:00,9887,Торговля,Чистая зона ВВЛ-вылет,191520.0
3283,8998,2022-05-31 23:00:00,7998,Торговля,Чистая зона ВВЛ-вылет,334800.0
8959,8998,2022-05-31 23:00:00,9887,Торговля,Чистая зона ВВЛ-вылет,215840.0
2867,8998,2022-05-31 23:30:00,7998,Торговля,Чистая зона ВВЛ-вылет,374800.0


In [121]:
print(data.groupby("sell_point").agg({"cashier": "nunique"}).sum())
data.groupby("sell_point").agg({"cashier": "nunique"})

cashier    11
dtype: int64


,cashier
sell_point,
0601,1
2373,1
2968,1
3726,1
5648,2
8313,3
8998,2


In [122]:
len(data["cashier"].unique())

11